In [1]:
import pandas as pd

# Load Excel file
file_path = "Analytics Position Case Study.xlsx"

# Load each sheet
gameplay_df = pd.read_excel(file_path, sheet_name="User Gameplay data")
deposit_df = pd.read_excel(file_path, sheet_name="Deposit Data")
withdrawal_df = pd.read_excel(file_path, sheet_name="Withdrawal Data")

# Display first 5 rows and first 5 columns from each sheet
print("🎮 User Gameplay Data")
display(gameplay_df.iloc[:, :5].head())

print("💰 Deposit Data")
display(deposit_df.iloc[:, :5].head())

print("🏦 Withdrawal Data")
display(withdrawal_df.iloc[:, :5].head())


🎮 User Gameplay Data


,User ID,Games Played,Datetime
0,851,1,2022-01-10 00:00:00
1,717,1,2022-01-10 00:00:00
2,456,1,2022-01-10 00:00:00
3,424,1,2022-01-10 00:00:00
4,845,1,2022-01-10 00:00:00


💰 Deposit Data


,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500


🏦 Withdrawal Data


,User Id,Datetime,Amount
0,190,2022-01-10 00:03:00,5872
1,159,2022-01-10 00:16:00,9540
2,164,2022-01-10 00:24:00,815
3,946,2022-01-10 00:29:00,23000
4,763,2022-01-10 00:40:00,9473


In [ ]:
Part A

In [35]:
import pandas as pd
from functools import reduce
from datetime import date

# Load Excel file
excel_path = 'Analytics Position Case Study 1.xlsx'
xls = pd.ExcelFile(excel_path)

# Load sheets
gameplay_df = xls.parse('User Gameplay data')
deposit_df = xls.parse('Deposit Data')
withdrawal_df = xls.parse('Withdrawal Data')

# Convert datetime and assign Date and Slot
for df in [gameplay_df, deposit_df, withdrawal_df]:
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df['Date'] = df['Datetime'].dt.date
    df['Slot'] = df['Datetime'].dt.hour.apply(lambda h: 'S1' if h < 12 else 'S2')

# Aggregate each data source
games_agg = gameplay_df.groupby(['User ID', 'Date', 'Slot'])['Games Played'].sum().reset_index()

deposits_agg = deposit_df.groupby(['User Id', 'Date', 'Slot']).agg(
    deposit_amount=('Amount', 'sum'),
    num_deposits=('Amount', 'count')
).reset_index().rename(columns={'User Id': 'User ID'})

withdrawals_agg = withdrawal_df.groupby(['User Id', 'Date', 'Slot']).agg(
    withdrawal_amount=('Amount', 'sum'),
    num_withdrawals=('Amount', 'count')
).reset_index().rename(columns={'User Id': 'User ID'})

# Merge all into one dataset
loyalty_data = reduce(lambda left, right: pd.merge(left, right, on=['User ID', 'Date', 'Slot'], how='outer'),
                      [games_agg, deposits_agg, withdrawals_agg])
loyalty_data.fillna(0, inplace=True)

# Calculate loyalty points
loyalty_data['loyalty_points'] = (
    0.01 * loyalty_data['deposit_amount'] +
    0.005 * loyalty_data['withdrawal_amount'] +
    0.001 * (loyalty_data['num_deposits'] - loyalty_data['num_withdrawals']).clip(lower=0) +
    0.2 * loyalty_data['Games Played']
)

# Extract separate tables for each requested slot

# a. 2nd October Slot S1
slot_a = loyalty_data[(loyalty_data['Date'] == date(2022, 10, 2)) & (loyalty_data['Slot'] == 'S1')][
    ['User ID', 'loyalty_points']
].sort_values(by='loyalty_points', ascending=False).reset_index(drop=True)
slot_a.columns = ['User ID', 'Loyalty Points (2nd Oct - S1)']

# b. 16th October Slot S2
slot_b = loyalty_data[(loyalty_data['Date'] == date(2022, 10, 16)) & (loyalty_data['Slot'] == 'S2')][
    ['User ID', 'loyalty_points']
].sort_values(by='loyalty_points', ascending=False).reset_index(drop=True)
slot_b.columns = ['User ID', 'Loyalty Points (16th Oct - S2)']

# c. 18th October Slot S1
slot_c = loyalty_data[(loyalty_data['Date'] == date(2022, 10, 18)) & (loyalty_data['Slot'] == 'S1')][
    ['User ID', 'loyalty_points']
].sort_values(by='loyalty_points', ascending=False).reset_index(drop=True)
slot_c.columns = ['User ID', 'Loyalty Points (18th Oct - S1)']

# d. 26th October Slot S2
slot_d = loyalty_data[(loyalty_data['Date'] == date(2022, 10, 26)) & (loyalty_data['Slot'] == 'S2')][
    ['User ID', 'loyalty_points']
].sort_values(by='loyalty_points', ascending=False).reset_index(drop=True)
slot_d.columns = ['User ID', 'Loyalty Points (26th Oct - S2)']

# Display all four slot tables
print("🔹 Loyalty Points - 2nd October S1")
display(slot_a)

print("\n🔹 Loyalty Points - 16th October S2")
display(slot_b)

print("\n🔹 Loyalty Points - 18th October S1")
display(slot_c)

print("\n🔹 Loyalty Points - 26th October S2")
display(slot_d)


🔹 Loyalty Points - 2nd October S1


,User ID,Loyalty Points (2nd Oct - S1)



🔹 Loyalty Points - 16th October S2


,User ID,Loyalty Points (16th Oct - S2)
0,634,1491.555
1,212,999.991
2,99,980.002
3,28,900.004
4,566,880.203
...,...,...
589,526,0.200
590,533,0.200
591,536,0.200
592,841,0.200



🔹 Loyalty Points - 18th October S1


,User ID,Loyalty Points (18th Oct - S1)
0,634,2723.100
1,208,1701.401
2,673,900.801
3,162,770.000
4,245,750.000
...,...,...
617,443,0.200
618,444,0.200
619,446,0.200
620,818,0.200



🔹 Loyalty Points - 26th October S2


,User ID,Loyalty Points (26th Oct - S2)
0,714,2000.001
1,369,1501.915
2,634,1237.010
3,538,1200.403
4,2,900.002
...,...,...
623,766,0.200
624,767,0.200
625,385,0.200
626,185,0.200


In [61]:

october_data = loyalty_data[
    (loyalty_data['Date'] >= date(2022, 10, 1)) &
    (loyalty_data['Date'] <= date(2022, 10, 31))
]

oct_summary = october_data.groupby('User ID').agg(
    total_loyalty_points=('loyalty_points', 'sum'),
    total_games_played=('Games Played', 'sum')
).reset_index()

oct_summary = oct_summary.sort_values(
    by=['total_loyalty_points', 'total_games_played'],
    ascending=[False, False]
).reset_index(drop=True)

oct_summary['rank'] = oct_summary.index + 1

print("Top 10 Ranked Players for October:")
display(oct_summary.head(10))


Top 10 Ranked Players for October:


,User ID,total_loyalty_points,total_games_played,rank
0,634,61121.160,22.0,1
1,714,14790.824,4.0,2
2,212,13947.218,0.0,3
3,672,13238.625,8.0,4
4,99,12469.951,4.0,5
5,566,12121.383,106.0,6
6,369,11137.117,22.0,7
7,740,11006.888,2.0,8
8,30,10802.562,9.0,9
9,365,10389.779,2368.0,10


In [51]:
avg_deposit_per_txn = deposit_df['Amount'].mean()
print(f"Average Deposit Amount per transaction: ₹{avg_deposit_per_txn:.2f}")

Average Deposit Amount per transaction: ₹5492.19


In [55]:
# Add Month column
deposit_df['Month'] = deposit_df['Datetime'].dt.to_period('M')

# Group by User + Month and sum deposits
monthly_deposit = deposit_df.groupby(['User Id', 'Month'])['Amount'].sum().reset_index()

# Now average across all user-months
avg_deposit_per_user_month = monthly_deposit['Amount'].mean()
print(f"Average Deposit Amount per User in a Month: ₹{avg_deposit_per_user_month:.2f}")

Average Deposit Amount per User in a Month: ₹22026.85


In [59]:
games_per_user = gameplay_df.groupby('User ID')['Games Played'].sum()
avg_games_per_user = games_per_user.mean()
print(f"Average Number of Games Played per User: {avg_games_per_user:.2f}")

Average Number of Games Played per User: 355.27


Part B

In [63]:
october_data = loyalty_data[
    (loyalty_data['Date'] >= date(2022, 10, 1)) &
    (loyalty_data['Date'] <= date(2022, 10, 31))
]

oct_summary = october_data.groupby('User ID').agg(
    total_loyalty_points=('loyalty_points', 'sum'),
    total_games_played=('Games Played', 'sum')
).reset_index()

# Rank players
oct_summary = oct_summary.sort_values(
    by=['total_loyalty_points', 'total_games_played'],
    ascending=[False, False]
).reset_index(drop=True)

oct_summary['rank'] = oct_summary.index + 1

# Filter top 50 players
top_50 = oct_summary.head(50).copy()

# Calculate bonus per player (proportional to loyalty points)
total_loyalty_top50 = top_50['total_loyalty_points'].sum()
bonus_pool = 50000

top_50['bonus_amount'] = top_50['total_loyalty_points'] / total_loyalty_top50 * bonus_pool
top_50['bonus_amount'] = top_50['bonus_amount'].round(2)

# Step 4: Display top 10
print("Top 10 Bonus Allocations:")
display(top_50[['User ID', 'total_loyalty_points', 'total_games_played', 'rank', 'bonus_amount']].head(10))


Top 10 Bonus Allocations:


,User ID,total_loyalty_points,total_games_played,rank,bonus_amount
0,634,61121.160,22.0,1,7240.96
1,714,14790.824,4.0,2,1752.25
2,212,13947.218,0.0,3,1652.31
3,672,13238.625,8.0,4,1568.37
4,99,12469.951,4.0,5,1477.30
5,566,12121.383,106.0,6,1436.01
6,369,11137.117,22.0,7,1319.40
7,740,11006.888,2.0,8,1303.98
8,30,10802.562,9.0,9,1279.77
9,365,10389.779,2368.0,10,1230.87


In [67]:
from datetime import date

# Filter only October data
october_data = loyalty_data[
    (loyalty_data['Date'] >= date(2022, 10, 1)) &
    (loyalty_data['Date'] <= date(2022, 10, 31))
]

# Summarize per user: loyalty points and games played
oct_summary = october_data.groupby('User ID').agg(
    total_loyalty_points=('loyalty_points', 'sum'),
    total_games_played=('Games Played', 'sum')
).reset_index()

# Sort and rank
oct_summary = oct_summary.sort_values(
    by=['total_loyalty_points', 'total_games_played'],
    ascending=[False, False]
).reset_index(drop=True)

oct_summary['rank'] = oct_summary.index + 1

# Select top 50
top_50 = oct_summary.head(50).copy()

# Calculate bonus share based on loyalty points
bonus_pool = 50000
total_loyalty_top50 = top_50['total_loyalty_points'].sum()

top_50['bonus_amount'] = (top_50['total_loyalty_points'] / total_loyalty_top50) * bonus_pool
top_50['bonus_amount'] = top_50['bonus_amount'].round(2)

# Display or export
print(" Top 50 Players Bonus Distribution")
display(top_50[['User ID', 'total_loyalty_points', 'total_games_played', 'rank', 'bonus_amount']])


 Top 50 Players Bonus Distribution


,User ID,total_loyalty_points,total_games_played,rank,bonus_amount
0,634,61121.160,22.0,1,7240.96
1,714,14790.824,4.0,2,1752.25
2,212,13947.218,0.0,3,1652.31
3,672,13238.625,8.0,4,1568.37
4,99,12469.951,4.0,5,1477.30
5,566,12121.383,106.0,6,1436.01
6,369,11137.117,22.0,7,1319.40
7,740,11006.888,2.0,8,1303.98
8,30,10802.562,9.0,9,1279.77
9,365,10389.779,2368.0,10,1230.87


Part C

In [ ]:

Yes, the loyalty point formula can be improved to make it more balanced and fair across all player types.

Suggestions:
1. Use logarithmic scaling for deposit/withdrawal amounts — this prevents high-value players from dominating.
   Example: 0.01 * log(1 + deposit amount)

2. Increase the weight of (#Deposits - #Withdrawals) from 0.001 to 0.01 — this better rewards financially positive behavior.

3. Add consistency/streak bonuses — reward users who play or deposit regularly over the month.

4. Adjust game weight by value or session time — a ₹1000 game shouldn't earn the same as a ₹10 game.

5. Add a daily point cap — to prevent players from exploiting the system with one-day bulk actions.

Overall, these changes would encourage sustained engagement, fairness, and long-term user retention.
